In [1]:
import os
import sys

sys.path.append(os.path.abspath('../..'))
import argparse
from sfl.utils.args import FLConfig
from sfl.utils.exp import get_model_and_tokenizer

config = FLConfig(
    collect_intermediates=False,
    global_round=10,
    client_evaluate_freq=500,
    client_epoch=1,  # 每轮联邦每个Client训2轮
    split_point_1=6,
    split_point_2=30,  # [0,1 | 2,3,.... 29| 30, 31]
    use_lora_at_trunk=True,  # 在trunk部分使用LoRA
    use_lora_at_top=True,
    use_lora_at_bottom=True,
    use_lora_at_embed=True,
    top_and_bottom_from_scratch='False',
    attack_mode=None,
    client_steps=700
)

args = {
    'dataset_train_frac': 1.0,
    'dataset_test_frac': 0.1,
    'dataset': 'piqa',
    'model_name': 'llama2',
    'save_checkpoint': True,
    'task_type': 'lm',
    'attacker_freq': 10,
    'attacker_samples':2,
    'log_to_wandb': False
}
# convert to namespace
args = argparse.Namespace(**args)

[2024-07-10 18:19:02,928] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     UNEXPECTED_EOF_WHILE_READING] EOF occurred in
[nltk_data]     violation of protocol (_ssl.c:1006)>


In [3]:
from torch.nn import Linear
from torch import nn
import  sys
import os
sys.path.append(os.path.abspath('../..'))
import argparse
from sfl.utils.args import FLConfig

# md,tok = get_model_and_tokenizer('gpt2')
#
# md.config_sfl(config)
class MLP(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ln = Linear(3, 2)

    def forward(self, x):
        return self.ln(x)


md = MLP()

In [4]:
content = {'model':md.state_dict()}

In [5]:
def transform_to_list(x):
        if isinstance(x, list) or isinstance(x, tuple):
            return [transform_to_list(each_x) for each_x in x]
        elif isinstance(x, dict):
            for key in x.keys():
                x[key] = transform_to_list(x[key])
            # x['order'] = '::'.join(list(x.keys()))
            return x
        else:
            if hasattr(x, 'tolist'):
               return x.tolist()
            else:
                return x

In [6]:
content1 = transform_to_list(content)

In [7]:
from experiments.notebook import gRPC_comm_manager_pb2

splited_msg = gRPC_comm_manager_pb2.MessageRequest()  # map/dict

In [13]:
def create_by_type(value, nested=False):
    if isinstance(value, dict):
        if isinstance(list(value.keys())[0], str):
            m_dict = gRPC_comm_manager_pb2.mDict_keyIsString()
            key_type = 'string'
        else:
            m_dict = gRPC_comm_manager_pb2.mDict_keyIsInt()
            key_type = 'int'
        for key in value.keys():
            m_dict.dict_value[key].MergeFrom(
                create_by_type(value[key], nested=True))
        if nested:
            msg_value = gRPC_comm_manager_pb2.MsgValue()
            if key_type == 'string':
                msg_value.dict_msg_string_key.MergeFrom(m_dict)
            else:
                msg_value.dict_msg_int_key.MergeFrom(m_dict)
            return msg_value
        else:
            return m_dict
    elif isinstance(value, list) or isinstance(value, tuple):
        m_list = gRPC_comm_manager_pb2.mList()
        for each in value:
            m_list.list_value.append(create_by_type(each,
                                                         nested=True))
        if nested:
            msg_value = gRPC_comm_manager_pb2.MsgValue()
            msg_value.list_msg.MergeFrom(m_list)
            return msg_value
        else:
            return m_list
    else:
        m_single = gRPC_comm_manager_pb2.mSingle()
        if type(value) in [int, np.int32]:
            m_single.int_value = value
        elif type(value) in [str, bytes]:
            m_single.str_value = value
        elif type(value) in [float, np.float32]:
            m_single.float_value = value
        else:
            raise ValueError(
                f'The data type {type(value)} has not been supported.')

        if nested:
            msg_value = gRPC_comm_manager_pb2.MsgValue()
            msg_value.single_msg.MergeFrom(m_single)
            return msg_value
        else:
            return m_single

In [14]:
def build_msg_value(value):
    msg_value = gRPC_comm_manager_pb2.MsgValue()

    if isinstance(value, list) or isinstance(value, tuple):
        msg_value.list_msg.MergeFrom(create_by_type(value))
    elif isinstance(value, dict):
        if isinstance(list(value.keys())[0], str):
            msg_value.dict_msg_string_key.MergeFrom(
                create_by_type(value))
        else:
            msg_value.dict_msg_int_key.MergeFrom(create_by_type(value))
    else:
        msg_value.single_msg.MergeFrom(create_by_type(value))

    return msg_value


In [18]:
x = build_msg_value(
            content)

AttributeError: dict_msg_string_key

In [15]:
splited_msg.msg['content'].MergeFrom()

AttributeError: dict_msg_string_key

In [ ]:
md.print_split_model()

In [ ]:
from sfl.utils.model import generate, get_output

get_output("### User:Hi, what", tok, md)

In [ ]:
# md.print_split_model()
md_lr = md.convert_to_lora_model()

In [ ]:
md_lr.print_split_model()

In [ ]:
for nm,p in md_lr.named_parameters():
    print(nm,p.size())

In [ ]:
from sfl.config import DRAConfig
from sfl.utils.exp import get_dra_attacker

atk_cfg = DRAConfig(target_model_name='llama2', target_dataset='sanitized', target_sps='6-6', train_label='val', target_model_load_bits=4)
atk, _ = get_dra_attacker(atk_cfg)

In [ ]:

from transformers import AdamW
from sfl.utils.model import evaluate_attacker_rouge
from sfl.utils.exp import get_dataset_class
from tqdm import tqdm_notebook

client_ids = ['0']

dataset_cls = get_dataset_class('sanitized')
dataset = dataset_cls(tokenizer=tokenizer, client_ids=client_ids)
test_loader = dataset.get_dataloader_unsliced(2, 'train', shrink_frac=0.2)

avg_rouge = 0
avg_rouge_dlg = 0
step = 0
opt = AdamW(model.parameters(), lr=1e-5)
config.noise_mode = 'dxp'
config.noise_scale = 1000.0
model.config_sfl(config)

dlg.to(model.device)
atk.to(model.device)
with tqdm_notebook(total=len(test_loader)) as pbar:
  for batch in test_loader:
    opt.zero_grad()
    input_ids = batch['input_ids'].to(model.device)
    o1 = model(input_ids, batch['attention_mask'].to(model.device), labels=input_ids)
    # o2 = dlg(tr2t.fx.to(model.device))
    # print(o1)
    o1.loss.backward()
    b2tr, tr2t, all = model.get_all_inter()
    opt.step()
    pred = atk(b2tr.fx.to(model.device))
    # print(batch['input_text'][0])
    gt = dlg.fit(tr2t.fx.to(model.device), tr2t.grad.to(model.device), epochs=20, gt_init=pred)
    # gt_texts = [tokenizer.decode(g.argmax(-1), skip_special_tokens=True) for g in gt]
    avg_rouge += evaluate_attacker_rouge(tokenizer, pred, batch)['rouge-l']['f']
    avg_rouge_dlg += evaluate_attacker_rouge(tokenizer, gt, batch)['rouge-l']['f']
    step += 1
    pbar.set_postfix({'dra_rouge': avg_rouge / step, 'dlg_rouge':avg_rouge_dlg/step})
    pbar.update()


In [ ]:
from sfl.utils.exp import add_sfl_params
import argparse
from typing import Any
from sfl.utils.model import Intermediate
from sfl.simulator.strategy import BaseSFLStrategy
from sfl.simulator.simulator import SFLSimulator

parser = argparse.ArgumentParser()
add_sfl_params(parser)
args = parser.parse_args({})

args.log_to_wandb = False
args.dlg_epochs = 30
args.dlg_init_with_dra = True

# 定义Client本地学习策略
class QAFLStrategy(BaseSFLStrategy):


    def sample_attacker_triggered(self, global_round, client_id, local_epoch, local_step,
                                  b2tr_inter: Intermediate, tr2t_inter: Intermediate,
                                  all_inter: dict[Any, Intermediate],
                                  batch, logs):
        encoder_inter = all_inter.get('encoder', None)
        with torch.no_grad():
            for type, atk in zip(['b2tr', 'tr2t'], [self.dra1, self.dra2]):
                if atk is None:
                    continue
                atk.to(self.simulator.device)
                inter = b2tr_inter if type == 'b2tr' else tr2t_inter
                if self.llm.type == 'encoder-decoder':
                    attacked = atk(torch.concat([encoder_inter.fx.to(
                        self.simulator.device), inter.fx.to(atk.device)], dim=1))
                else:
                    attacked = atk(inter.fx.to(atk.device))
                rouge_res = calculate_rouge(self.tokenizer, attacked, batch['input_text'])
                self.log_to_sample_result(client_id, f'attacker_{type}', rouge_res['rouge-l']['f'])
                self.log_to_all_result(client_id, f'attacker_{type}', rouge_res['rouge-l']['f'])
                logs[f'attacker_{type}_step'] = rouge_res['rouge-l']['f']
        gt_init = None
        if self.args.dlg_init_with_dra:
            gt_init = attacked
        self.dlg.to(self.simulator.device)
        gt = self.dlg.fit(tr2t_inter.fx.to(self.simulator.device), tr2t_inter.grad.to(self.simulator.device),
                          epochs=self.args.dlg_epochs,
                          adjust=False,
                          beta=self.args.dlg_beta,
                          gt_init=gt_init,
                          gt_reg=self.args.dlg_dra_reg,
                          temp_range=self.args.dlg_temp_range,
                          further_ft=self.args.dlg_further_ft,
                          encoder_inter=None if encoder_inter is None else encoder_inter.fx.to(
                              self.simulator.device)
                          )
        if self.llm.type == 'encoder-decoder':
            # replace the latter half of attacked to gt
            attacked[:, -gt.shape[1]:, :] = gt
            rouge = calculate_rouge(self.tokenizer, attacked, batch['input_text'])
        else:
            rouge = calculate_rouge(self.tokenizer, gt, batch['input_text'])
        self.log_to_sample_result(client_id, 'tag_rouge_lf', rouge['rouge-l']['f'])
        self.log_to_all_result(client_id, 'tag_rouge_lf', rouge['rouge-l']['f'])
        print(self.attack_all_performs)



simulator = SFLSimulator(client_ids=client_ids,
                             strategy=QAFLStrategy(args, model, tokenizer, test_loader, atk, None,dlg),
                             llm=model,
                             tokenizer=tokenizer,
                             dataset=dataset, config=config, args=args)

In [ ]:
simulator.simulate()

# 生成SensMarked数据集

In [ ]:
import pandas as pd
raw = pd.read_csv('/home/project/SFL-LLM/sanitized_data.csv')

In [ ]:
raw

In [ ]:
import re
import ast
from tqdm import tqdm_notebook

import pandas as pd
raw = pd.read_csv('/home/project/SFL-LLM/sanitized_data.csv')
# take only 'content' and 'entity'
raw = raw[['content', 'entity', 'sani_gpt4','sani_label']]


marked_content = []

for i, row in tqdm_notebook(raw.iterrows(), total=len(raw)):
    # get the content and entity
    data = {}
    content = row['content']
    entity = row['entity']
    data['sentence'] = content
    # print(entity)
    entity = ast.literal_eval(entity)
    replaced_places = []
    for e in entity:
        indexes = [(m.start(), m.end()) for m in re.finditer(re.escape(e), content)]
        for idx in indexes:
            if any([idx[0] > r[0] and idx[1] < r[1] for r in replaced_places]):
                continue
            content = content[:idx[0]] + '<P>' + content[idx[0]:idx[1]] + '<\P>' + content[idx[1]:]
            replaced_places.append(idx)
    marked_content.append(content)

df = pd.DataFrame(marked_content, columns=['marked_content'])
new = pd.concat([raw, df], axis=1)

In [ ]:
# split the dataset
type_labels = ['train'] * len(new)
# randomly select 20% indexes
all_indexes = list(range(len(new)))
import random

test_indexes = random.sample(all_indexes, int(len(new) * 0.25))
for i in test_indexes:
    type_labels[i] = 'test'

all_indexes = list(set(all_indexes) - set(test_indexes))
val_indexes = random.sample(all_indexes, int(len(new) * 0.15))
for i in val_indexes:
    type_labels[i] = 'validation'

# make type_labels to dataframe and concat it with the original dataframe
df = pd.DataFrame(type_labels, columns=['type'])
# concat it with new
new = pd.concat([new, df], axis=1)

In [ ]:
new.to_csv('/home/project/SFL-LLM/sensi.csv', index=False)

In [ ]:


from datasets import Dataset



dataset = Dataset.from_pandas(new)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../..'))

# _, tokenizer = get_model_and_tokenizer('bert')
# model, t = get_model_and_tokenizer('flan-t5-large')

In [ ]:

from sfl.simulator.dataset import FedDataset


class SanitizedFedDataset(FedDataset):

    def _format(self, example):
        return {'input': example['content'], 'entities': ast.literal_eval(example['entity'])}

    def _col_fun(self, batch):
        texts = [b['input'] for b in batch]
        input = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        mask = torch.zeros_like(input['input_ids'])
        for sp, sample in enumerate(batch):
            seq = input['input_ids'][sp].numpy().tolist()
            r = tokenizer(sample['entities'], add_special_tokens=False)
            for subseq in r.input_ids:
                for i in range(len(seq) - len(subseq) + 1):
                    if seq[i:i + len(subseq)] == subseq:
                        mask[sp, i:i + len(subseq)] = 1

        return {'input_ids': input['input_ids'],
                'attention_mask': input['attention_mask'],
                'input_text': texts, 'entities': [b['entity'] for b in batch],
                'input_santi_mask': mask}

    def __init__(self, tokenizer, client_ids: list[str], ):
        self.df = pd.read_csv('/home/project/SFL-LLM/sanitized_data_marked.csv')
        dataset = {
            'train': Dataset.from_pandas(self.df[self.df['type'] == 'train']),
            'val': Dataset.from_pandas(self.df[self.df['type'] == 'val']),
            'test': Dataset.from_pandas(self.df[self.df['type'] == 'test'])
        }
        super().__init__(tokenizer, client_ids, dataset, ['train', 'val', 'test'])


ds = SanitizedFedDataset(tokenizer, ['0'])

In [ ]:
ld = ds.get_dataloader_unsliced(6, 'val')

In [ ]:

for batch in ld:
    # find input_ids masked by mask
    input_ids = batch['input_ids']
    mask = batch['input_santi_mask']
    masked = input_ids * mask

    print(tokenizer.decode(masked[0],skip_special_tokens=True))
    print(batch['entities'][0])
    break

In [ ]:
import os
import sys
sys.path.append(os.path.abspath('../'))
from sfl.utils.exp import get_model_and_tokenizer
from sfl.utils.args import FLConfig
from sfl.utils.model import get_best_gpu

model, processor = get_model_and_tokenizer('vit-large')

device = get_best_gpu()
model.to(device)
config = FLConfig(
    collect_intermediates=False,
    global_round=10,
    client_evaluate_freq=500,
    client_epoch=1,  # 每轮联邦每个Client训2轮
    split_point_1=6,
    split_point_2=20,  # [0,1 | 2,3,.... 29| 30, 31]
    use_lora_at_trunk=False,  # 在trunk部分使用LoRA
    use_lora_at_top=False,
    use_lora_at_bottom=False,
    top_and_bottom_from_scratch='True',
    attack_mode='b2tr',
    client_steps=700
)

model.config_sfl(config, None)

In [ ]:
from sfl.utils.exp import get_dataset

ds = get_dataset('imagewoof',processor, client_ids=['0'], shrink_frac=0.1)
dl = ds.get_dataloader_unsliced(64,'train', 1.0)

In [ ]:
from sfl.model.attacker.dra_attacker import ViTDRAttacker
from sfl.model.attacker.dra_attacker import ViTDRAttackerConfig

attacker = ViTDRAttacker(ViTDRAttackerConfig(), model.config)

In [ ]:
from sfl.utils.model import convert_to_image
# train the attacker
import torch
import torch.optim as optim
from tqdm import tqdm_notebook

# attacker.to(device)

def test(md, atk, image):
    atk.to(md.device)
    inter = md(image['input'].to(model.device))
    image = convert_to_image(atk(inter))
    image[0].show()

In [ ]:
optimizer = optim.AdamW(attacker.parameters(), lr=1e-3, weight_decay=1e-5)

epochs = 10

with tqdm_notebook(total=epochs*len(dl)) as pbar:
    step = 0
    for epc in range(epochs):
        for batch in dl:
            input_tensor = batch['input'].to(device)
            inter = model(input_tensor)
            recovered = attacker(inter)
            loss = torch.nn.functional.mse_loss(recovered, input_tensor)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            pbar.set_description(f'Epoch {epc} Step {step} Loss {loss.item()}')
            pbar.update(1)
            step += 1
            # if step % 100 == 0:
            #     test(model, attacker, sample)


In [ ]:

from sfl.utils.exp import get_dra_attacker, DRAConfig

atk, _ = get_dra_attacker(
    DRAConfig(target_model_name='vit-large', larger_better=False, target_sps='6-999', train_label='validation',
              dataset='imagewoof', model='vit', tr2t_enable=False,prefix='gaussian:0.0001'))

In [ ]:

test_dl = ds.get_dataloader_unsliced(3, 'train')
num = 3
for sample in test_dl:
    test(model, atk, sample)
    num -= 1
    if num == 0:
        break

In [ ]:
import os
import sys

sys.path.append(os.path.abspath('../..'))

from sfl.utils.exp import get_dataset, get_tokenizer

tok = get_tokenizer('gpt2-large')
tok.pad_token = tok.eos_token
data = get_dataset('piqa',tok,[])
dl = data.get_dataloader_unsliced(10,'test',1.0)

In [ ]:
from sfl.utils.model import get_best_gpu
device = get_best_gpu()

md.to(device)

In [ ]:
from sfl.utils.model import generate

generate("User: does the equation x^2+y^2=10 hold if x=2 and y=4? , Assistant: ",tok, md)

In [ ]:
import torch
import numpy as np
tensor = np.array([[0.1, 0.3, 0.4, 0.2, 0.6]])
bins = [0.1, 0.3, 0.5]

np.digitize(tensor, bins)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../..'))
from sfl.utils.exp import get_model_and_tokenizer


md, tok = get_model_and_tokenizer('vicuna',load_bits=32)

In [ ]:
from sfl.utils.model import generate

res = generate('### User: given the following EHR, predict the mortality and give your reason,'
               'Name|Gender|Length-of-stay|GCS|Heart Rate|Ethnicity|Age|\n'
               'Sam|Male|90000hours|43.2|102|Black|69|\n'
         '### Assistant: The probability of mortality, and the reasons are:', tok, md)
print(res)

In [2]:
import torch
import numpy as np
tensor = np.array([[0.1, 0.3, 0.4, 0.2, 0.6]])
bins = [0.1, 0.3, 0.5]

np.digitize(tensor, bins)

array([[1, 2, 2, 1, 3]])

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../..'))
from sfl.utils.exp import get_model_and_tokenizer


md, tok = get_model_and_tokenizer('vicuna',load_bits=32)

[nltk_data] Downloading package wordnet to
[nltk_data]     /data/stupidtree/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from sfl.utils.model import generate

res = generate('### User: given the following EHR, predict the mortality and give your reason,'
               'Name|Gender|Length-of-stay|GCS|Heart Rate|Ethnicity|Age|\n'
               'Sam|Male|90000hours|43.2|102|Black|69|\n'
         '### Assistant: The probability of mortality, and the reasons are:', tok, md)
print(res)

### User: given the following EHR, predict the mortality and give your reason,Name|Gender|Length-of-stay|GCS|Heart Rate|Ethnicity|Age|
Sam|Male|90000hours|43.2|102|Black|69|
### Assistant: The probability of mortality, and the reasons are:

| Name | Gender | Length-of-stay | GCS | Heart Rate | Ethnicity | Age |
| --- | --- | --- | --- | --- | --- | --- |
| Sam | Male | 90000hours | 43.2 | 102 | Black | 69 |

The probability of mortality for this patient is 0.15, which means there is a 15% chance of death within the next year.

The reasons for the high mortality probability are:

1. The patient's age (69 years old) is a significant risk factor for mortality.
2. The patient's GCS (43.2) is low, indicating a significant level of brain dysfunction, which can lead to poor outcomes and increased mortality risk.
3. The patient's heart rate (102) is high, which can indicate various cardiovascular conditions and increase the risk of mortality.
4. The patient's ethnicity (Black) is a risk factor

In [1]:
import pandas as pd
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("stupidtree/[CR]BiSR(b+f)")

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })


In [21]:
import numpy as np
model = 'llama2'
metric = 'METEOR'
methods = ['SIP_b2tr','BiSR(b)','BiSR(f)','BiSR(b+f)']
all_datasets = runs_df['config'].apply(lambda x: x['dataset']).unique()

In [36]:
# select where config['model_name'] == model
df_model = runs_df[runs_df['config'].apply(lambda x: x['model_name'] == model)]

for method in methods:
    print(method, end='\t')
    for dataset in all_datasets:
        df = df_model[df_model['config'].apply(lambda x: x['dataset'] == dataset)]
        seeds = df['config'].apply(lambda x: x['seed']).unique()
        ress = []
        for seed in seeds:
            df_seed = df[df['config'].apply(lambda x: x['seed'] == seed)]
            key = f'client0_{method}_{metric}_avg'
            performance = df_seed['summary'].apply(lambda x: x[key]).mean()
            ress.append(performance*100)

        print(f'{np.mean(ress):.2f}$\pm${np.std(ress):.2f}\t',end='\t')
    print('\n')



SIP_b2tr	96.37$\pm$1.18		66.67$\pm$0.89		88.05$\pm$0.25		96.57$\pm$0.26		89.06$\pm$0.54		

BiSR(b)	97.42$\pm$0.76		85.65$\pm$0.83		93.80$\pm$1.03		97.10$\pm$0.32		94.65$\pm$0.32		

BiSR(f)	98.94$\pm$0.34		94.39$\pm$1.22		95.96$\pm$1.05		99.01$\pm$0.29		97.70$\pm$0.19		

BiSR(b+f)	99.47$\pm$0.42		96.58$\pm$0.14		97.71$\pm$0.06		99.46$\pm$0.14		98.03$\pm$0.17		

